<a href="https://colab.research.google.com/github/nataliechernysh/NEW_final/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("server_AHXTHQI46MJWDJ2TMOGZI3X5-CONLHEZHSSEZPNDK")

Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable #so it is available to call from our anvil 



SyntaxError: ignored

# Import Files


In [134]:
import numpy as np
import pandas as pd
from scipy import signal 
from matplotlib import pyplot as pyplot
%matplotlib inline
!pip install ipympl
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files;
df=files.upload();
filename = list(df.keys ())[-1]
ds= pd.ExcelFile(filename)
sheetnames=ds.sheet_names
datasheet=pd.read_excel(filename, sheet_name=0); 
!rm{filename}
datasheet
total_data_sheet = np.array(datasheet)

# Parse into dictionaries

In [ ]:
total_data_sheet = np.array(datasheet)

In [ ]:
def compile_accelerometer_data(total_data_sheet):
  #accelerometer data contains two data points per frame
  accelerometer_data = total_data_sheet[50626:54000,2:50]
  #print(accelerometer_data)
  temp_array = np.zeros(shape=(np.shape(accelerometer_data)[0]//2, 48))

  #goes through all of the accelerometer data and finds the average of each of
  #the two data points per frame
  count = 0
  dimensions = np.shape(accelerometer_data)

  for row in range(0,dimensions[0]-1,2):
    avg_array = np.sum(accelerometer_data[row:row+2,:],axis=0) / 2      
    temp_array[count] = avg_array
    count+=1
  accelerometer_data = temp_array

  #print(accelerometer_data[1][1])
  return accelerometer_data

In [ ]:
def compile_vicon_data(total_data_sheet):
  vicon_data = total_data_sheet[54006:,2:119]
  return vicon_data

In [ ]:
def compile_emg_data(total_data_sheet):
    #emg data contains ten data points per frame
    delsys_emg_data = total_data_sheet[4:33744,2:18]
    
    #print(delsys_emg_data)
    
    temp_array = np.zeros(shape=(np.shape(delsys_emg_data)[0]//20, 16))   
    #print(np.sum(delsys_emg_data[0:20,:],axis=0) / 20)
    count = 0
    dimensions = np.shape(delsys_emg_data)
    #goes through all of the accelerometer data and finds the average of each of
    #the ten data points per frame
    for row in range(0,dimensions[0]-1,20):
        avg_array = np.sum(delsys_emg_data[row:row+20,:],axis=0) / 20      
        temp_array[count] = avg_array
        count+=1
   
    delsys_emg_data = temp_array
    
    return delsys_emg_data

In [ ]:
def parse_emg_headers(total_data_sheet):

  emg_headers = total_data_sheet[2,2:18]

  return emg_headers

In [ ]:
def parse_accelerometer_headers(total_data_sheet):

  accelerometer_headers = total_data_sheet[50624,2:50]

  return accelerometer_headers


In [ ]:
def parse_vicon_dictionary(total_data_sheet):

  vicon_data = compile_vicon_data(total_data_sheet)

  column = 0
  coordinate_count = 0
  sub_headers = {}
  vicon_headers = ["LFHD","RFHD","LBDH","RBHD","C7","T10","CLAV","STRN","RBAK","LSHO","LUPA","LELB","LFRM","LWRA","LWRB","LFIN","RSHO","RUPA","RELB","RFRM","RWRA","RWRB","RFIN","LASI","RASI","LPSI","RPSI","LTHI","LKNE","LTIB","LANK","LHEE","LTOE","RTHI","RKNE","RTIB","RANK","RHEE","RTOE"]
  vicon_dictionary = {}

  for header in vicon_headers:
    marker = {}
    for coordinate_count in range(0,3):
      if coordinate_count == 0:
        marker['X'] = vicon_data[:,column]
        column+=1
      elif coordinate_count == 1:
        marker['Y'] = vicon_data[:,column]
        column+=1
      elif coordinate_count == 2:
        marker['Z'] = vicon_data[:,column]
        column+=1
    
    vicon_dictionary[header] = marker
  print(vicon_dictionary['LFHD'])
  return vicon_dictionary




In [ ]:
parse_vicon_dictionary(total_data_sheet)

In [ ]:
def parse_accelerometer_dictionary(total_data_sheet):
  accelerometer_headers = ["ACC1","ACC2","ACC3","ACC4","ACC5","ACC6","ACC7","ACC8","ACC9","ACC10","ACC11","ACC12","ACC13","ACC14","ACC15","ACC16"]
  accelerometer_data = compile_accelerometer_data(total_data_sheet)

  accelerometer_dictionary = {}
  column = 0
  for header in accelerometer_headers:
    marker = {}
    for coordinate_count in range(0,3):
      if coordinate_count == 0:
        marker['X'] = accelerometer_data[:,column]
        column+=1
      elif coordinate_count == 1:
        marker['Y'] = accelerometer_data[:,column]
        column+=1
      elif coordinate_count == 2:
        marker['Z'] = accelerometer_data[:,column]
        column+=1
    
    accelerometer_dictionary[header] = marker
  print(accelerometer_dictionary['ACC1'])

In [ ]:
parse_accelerometer_dictionary(total_data_sheet)


In [ ]:
def parse_emg_dictionary(total_data_sheet):
    emg_data = compile_emg_data(total_data_sheet)
    emg_headers = parse_emg_headers(total_data_sheet)
    count = 0
    emg_dictionary = {}
    for headers in emg_headers:
        emg_dictionary[headers] = emg_data[:, count]
        count += 1
    #print(emg_dict.keys())
    #print(emg_dict['EMG1'])

    return emg_dictionary

In [ ]:
def parse_vicon_to_dictionary(total_data_sheet):
  vicon_data = compile_vicon_data(total_data_sheet)
  vicon_headers = parse_vicon_dictionary(total_data_sheet)
  count = 0 
  vicon_dictionary = {}
  for headers in vicon_headers:
    vicon_dictionary[headers] = vicon_data[:, count]
    count += 1
  return vicon_dictionary

In [ ]:
emg_data = compile_emg_data(total_data_sheet)
emg_headers = parse_emg_headers(total_data_sheet)
emg_dict = parse_emg_dictionary(emg_data,emg_headers)

TypeError: ignored

In [ ]:

# create dictionaries of consecutive toe minima to define toe-off points in gait cycle
def find_toe_minima(vicon_dictionary):
    LTOE_dict = vicon_dictionary['LTOE']
    RTOE_dict = vicon_dictionary['RTOE']

    # find minima (using z-coordinates) of each marker
    LTOE_minima = signal.argrelmin(LTOE_dict['Z'], order=20)
    RTOE_minima = signal.argrelmin(RTOE_dict['Z'], order=20)

    # argrelmin output is tuple, create array of frames at which minima occur
    LTOE_minima = LTOE_minima[0]
    RTOE_minima = RTOE_minima[0]

    # create dictionaries of minima for each marker, with frames as keys
    L_mindict = {}
    R_mindict = {}

    # add minima to dictionaries
    for frame in LTOE_minima:
        L_mindict[frame] = LTOE_dict[frame]
    for frame in RTOE_minima:
        R_mindict[frame] = RTOE_dict[frame]

    return LTOE_minima, RTOE_minima

# Wire Frame and read connections

In [ ]:
#n corresponds to the frame you chose ... FIX THIS FROM BEING A GLOBAL VARIABLE!!! 
# list of connections FROM, TO for the wireframe model


def connect_coord(marker_1, marker_2, vicon_dictionary):
    segment = [
        [vicon_dictionary[marker_1][n, 0], vicon_dictionary[marker_2][n, 0]], #n corresponds to the frame you chose 
        [vicon_dictionary[marker_1][n, 1], vicon_dictionary[marker_2][n, 1]],
        [vicon_dictionary[marker_1][n, 2], vicon_dictionary[marker_2][n, 2]], 
    ]
    
    return segment

def build_model(vicon_dictionary):
    connections = [
      # head
      ('RFHD', 'LFHD'),
      ('LFHD', 'LBHD'),
      ('LBHD', 'RBHD'),
      ('RBHD', 'RFHD'),
      # upper body/trunk
      ('RSHO', 'CLAV'),
      ('CLAV', 'LSHO'),
      ('RSHO', 'LSHO'),
      ('CLAV', 'C7'),
      ('CLAV', 'STRN'),
      ('C7', 'T10'),
      ('STRN', 'T10'),
      # upper limbs
      ('RSHO', 'RUPA'),
      ('RUPA', 'RELB'),
      ('RELB', 'RFRM'),
      ('RFRM', 'RWRA'),
      ('RELB', 'RWRB'),
      ('RWRA', 'RWRB'),
      ('RWRA', 'RFIN'),
      ('RWRB', 'RFIN'),
      ('LSHO', 'LUPA'),
      ('LUPA', 'LELB'),
      ('LELB', 'LFRM'),
      ('LFRM', 'LWRA'),
      ('LELB', 'LWRB'),
      ('LWRA', 'LWRB'),
      ('LWRA', 'LFIN'),
      ('LWRB', 'LFIN'),
      # pelvis/hip
      ('LASI', 'RASI'),
      ('RASI', 'RPSI'),
      ('RPSI', 'LPSI'),
      ('LPSI', 'LASI'),
      # lower limbs
      ('RASI', 'RTHI'),
      ('RPSI', 'RTHI'),
      ('RASI', 'RKNE'),
      ('RPSI', 'RKNE'),
      ('RTHI', 'RKNE'),
      ('RKNE', 'RTIB'),
      ('RTIB', 'RANK'),
      ('RHEE', 'RANK'),
      ('RKNE', 'RANK'),
      ('RANK', 'RTOE'),
      ('RHEE', 'RTOE'),
      ('LASI', 'LTHI'),
      ('LPSI', 'LTHI'),
      ('LASI', 'LKNE'),
      ('LPSI', 'LKNE'),
      ('LTHI', 'LKNE'),
      ('LKNE', 'LTIB'),
      ('LTIB', 'LANK'),
      ('LHEE', 'LANK'),
      ('LKNE', 'LANK'),
      ('LANK', 'LTOE'),
      ('LHEE', 'LTOE')
    ]
    # create list of segments, each connecting 2 markers
    segments = []

    for m1, m2 in connections:
        segment = connect_coord(m1, m2, vicon_dictionary)
        segments.append(segment)

    return segments

def plot_model(vicon_dict, n):

    fig = pyplot.figure()
    ax = {
        'vicon': fig.add_subplot(1, 1, 1, projection='3d')
    } 

    # iterate through markers and plot individual coordinates
    for marker in markers:
        x, y, z = vicon_dictionary[marker][n, :]
        ax['vicon'].plot(x, y, z, marker='o', ms=3., markerfacecolor='k', color='k')
    
    # iterate through line segments list and plot connections
    for segment in segments:
        ax['vicon'].plot(segment[0], segment[1], segment[2], c='blue')
    
    # fix aspect ratio
    ax['vicon'].set_aspect('equal')

    # title figure
    ax['vicon'].set_title('youngbin_vicon: frame ' + str(n))

    # add axis labels
    ax['vicon'].set_xlabel('x (mm)')
    ax['vicon'].set_ylabel('y (mm)')
    ax['vicon'].set_zlabel('z (mm)')

    pyplot.show()



# Find steps using gait cycle function

In [ ]:
def find_heel_minima(vicon_dictionary):
    LHEE_dict = vicon_dictionary['LHEE']
    RHEE_dict = vicon_dictionary['RHEE']

    # find minima (using z-coordinates) of each marker
    LHEE_minima = signal.argrelmin(LHEE_dict['Z'], order=20)  #looking for order of z coordinates. The higher the order, the less minima it finds in that location 
    RHEE_minima = signal.argrelmin(RHEE_dict['Z'], order=20)

    # argrelmin output is tuple, create array of frames at which minima occur
    LHEE_minima = LHEE_minima[0] #describes the frames of the minima (when they occur)
    RHEE_minima = RHEE_minima[0]

    return LHEE_minima, RHEE_minima

def find_toe_minima(vicon_dictionary):
    LTOE_dict = vicon_dictionary['LTOE']
    RTOE_dict = vicon_dictionary['RTOE']

    # find minima (using z-coordinates) of each marker
    LTOE_minima = signal.argrelmin(LTOE_dict['Z'], order=20)
    RTOE_minima = signal.argrelmin(RTOE_dict['Z'], order=20)

    # argrelmin output is tuple, create array of frames at which minima occur
    LTOE_minima = LTOE_minima[0]
    RTOE_minima = RTOE_minima[0]

    return LTOE_minima, RTOE_minima
    print(LTOE_minima)


# Counting steps: Combine lists (heel and toe) Left (-1) and Right (+1)


In [ ]:
def combine_lists(list1, list2):
    combined_dict = {}

    # -1 is assigned to left marker
    for i in list1:
        combined_dict[i] = -1

    # +1 is assigned to right marker
    for i in list2:
        combined_dict[i] = 1

    # sort dictionary of step times into chronological order
    combined_dict = sorted(combined_dict.items())
    sorted_dict = sorted(combined_dict)
    combined_dict = dict(sorted_dict)
    # print(combined_dict.items())

    return combined_dict

def scan_dict(combined_dict): #looks for zeroes to determine pairing... if not zero, break current list 
    filtered_pairs = []
    filtered_pairs_temp = []
    keys_list = []

    for i in combined_dict.keys():
      keys_list.append(i)
    print('keys list', keys_list)     
# iterate through combined dictionary of L and R minima
# check if consecutive values are from opposite feet (-1 + 1 = 0)
# if sum of values is not 0, consecutive values are from the same foot
# print(keys_list)
    for i in range(0, len(keys_list) - 1):
      current_key = keys_list[i]
      next_key = keys_list[i + 1]
      sum = combined_dict[current_key] + combined_dict[next_key]
    # print(sum)

      if sum == 0:
        filtered_pairs_temp.append((current_key, next_key))
      # print(current_key,',', next_key)
      else:
          if len(filtered_pairs_temp) > 0:
              filtered_pairs.append(filtered_pairs_temp)
          filtered_pairs_temp = []

# print(filtered_pairs)
    return filtered_pairs #list of lists of pairs (applicable to both heel and toe markers )

def find_filtered_frames(filtered_pairs, combined_dict, vicon_dict, datatype=''):
    L_combined_list = []
    R_combined_list = []
    L_combined_temp = []
    R_combined_temp = []

    for item in filtered_pairs:
        if len(item) > 0:
            for i in range(0, len(item)):
                f1, f2 = item[i]
                if combined_dict[f1] == -1 and f1 not in L_combined_temp:
                    L_combined_temp.append(f1)
                if combined_dict[f1] == 1 and f1 not in R_combined_temp:
                    R_combined_temp.append(f1)
                if combined_dict[f2] == -1 and f2 not in L_combined_temp:
                    L_combined_temp.append(f2)
                if combined_dict[f2] == 1 and f2 not in R_combined_temp:
                    R_combined_temp.append(f2)

            L_combined_list.append(L_combined_temp)
            R_combined_list.append(R_combined_temp)

        L_combined_temp = []
        R_combined_temp = []

    # print(L_combined_list)
    # print(R_combined_list)

    L_gradients = np.gradient(vicon_dictionary['LTOE']['Z'])
    R_gradients = np.gradient(vicon_dictionary['RTOE']['Z'])
    L_double_gradients = np.gradient(L_gradients)
    R_double_gradients = np.gradient(R_gradients)

    # print(L_double_gradients)... not the exact frame of toe off but very close
    #double gradient refers to the second derivative–the maxes that represent toe off points. 
    #This is how you find the absolute minima for a given cycle (to differentiate from other toe minima in the area) (in numpy, the function is gradient) 
    L_double_grad_maxes = signal.argrelmax(L_double_gradients, order=20)
    L_double_grad_maxes = L_double_grad_maxes[0]
    R_double_grad_maxes = signal.argrelmax(R_double_gradients, order=20)
    R_double_grad_maxes = R_double_grad_maxes[0]  
  
    # print(L_double_grad_maxes)

    if datatype == 'toe':
        LTOE_zmin_frames = []
        RTOE_zmin_frames = []
        L_final_frames = []
        R_final_frames = []
        L_master_list = []
        R_master_list = []

        for i in range(0, len(L_combined_list)):
            current_list = L_combined_list[i]
            for frame in current_list:
                LTOE_zmin_frames.append(frame)
        for i in LTOE_zmin_frames:
            if i in L_double_grad_maxes:
                L_final_frames.append(i)
            elif i + 1 in L_double_grad_maxes:
                L_final_frames.append(i)
            elif i - 1 in L_double_grad_maxes:
                L_final_frames.append(i)
            elif i + 2 in L_double_grad_maxes:
                L_final_frames.append(i)
            elif i - 2 in L_double_grad_maxes:
                L_final_frames.append(i)

        for i in range(0, len(R_combined_list)):
            current_list = R_combined_list[i]
            for frame in current_list:
                LTOE_zmin_frames.append(frame)
        for i in LTOE_zmin_frames:
            if i in R_double_grad_maxes:
                R_final_frames.append(i)
            elif i + 1 in R_double_grad_maxes:
                R_final_frames.append(i)
            elif i - 1 in R_double_grad_maxes:
                R_final_frames.append(i)
            elif i + 2 in R_double_grad_maxes:
                R_final_frames.append(i)
            elif i - 2 in R_double_grad_maxes:
                R_final_frames.append(i)
        
        # print(L_final_frames)
        L_sublist_dict = {}
        for i in range(0, len(L_final_frames)):
            for m in range(0, len(L_combined_list)):
                current_list = L_combined_list[m]
                if L_final_frames[i] in current_list:
                    if m not in L_sublist_dict.keys():
                        L_sublist_dict[m] = []
                    L_sublist_dict[m].append(L_final_frames[i])
                    break
        # print(sublist_dict)
        for key in list(sorted(L_sublist_dict.keys())):
            L_master_list.append(L_sublist_dict[key])
        # print(L_master_list)

        R_sublist_dict = {}
        for i in range(0, len(R_final_frames)):
            for m in range(0, len(R_combined_list)):
                current_list = R_combined_list[m]
                if R_final_frames[i] in current_list:
                    if m not in R_sublist_dict.keys():
                        R_sublist_dict[m] = []
                    R_sublist_dict[m].append(R_final_frames[i])
                    break
        # print(sublist_dict)
        for key in list(sorted(R_sublist_dict.keys())):
            R_master_list.append(R_sublist_dict[key])

        L_combined_list = L_master_list
        R_combined_list = R_master_list

    print(L_combined_list)
    # print(R_combined_list)
    
    return L_combined_list, R_combined_list


In [ ]:
vicon_dictionary = parse_vicon_dictionary(total_data_sheet)
LHEE_minima, RHEE_minima = find_heel_minima(vicon_dictionary)
print(LHEE_minima)
#these are the frames for which the left heel is at a minima 


{'X': array([1179.567627, 1180.465088, 1183.190186, ..., 1100.72937,
       1099.951294, 1099.004639], dtype=object), 'Y': array([980.660767, 979.854248, 980.890991, ..., 1129.947754, 1129.679565,
       1131.239746], dtype=object), 'Z': array([1644.505615, 1643.941162, 1643.405151, ..., 1670.339478,
       1670.677612, 1671.902222], dtype=object)}
[   3  127  223  357  486  592  709  848  983 1009 1097 1158 1215 1361
 1652 1684]


/usr/local/lib/python3.10/dist-packages/scipy/signal/_peak_finding.py:75: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, plus)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_peak_finding.py:76: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, minus)


In [ ]:
vicon_dictionary = parse_vicon_dictionary(total_data_sheet)
LTOE_minima, RTOE_minima = find_toe_minima(vicon_dictionary)
heel_combined_dict = combine_lists(LHEE_minima, RHEE_minima)
toe_combined_dict = combine_lists(LTOE_minima, RTOE_minima)
heel_filtered_pairs = scan_dict(heel_combined_dict)
toe_filtered_pairs = scan_dict(toe_combined_dict)
LHEE_combined_list, RHEE_combined_list = find_filtered_frames(heel_filtered_pairs, heel_combined_dict, vicon_dictionary)
LTOE_combined_list, RTOE_combined_list = find_filtered_frames(toe_filtered_pairs, toe_combined_dict, vicon_dictionary, 'toe')
print(toe_filtered_pairs)

{'X': array([1179.567627, 1180.465088, 1183.190186, ..., 1100.72937,
       1099.951294, 1099.004639], dtype=object), 'Y': array([980.660767, 979.854248, 980.890991, ..., 1129.947754, 1129.679565,
       1131.239746], dtype=object), 'Z': array([1644.505615, 1643.941162, 1643.405151, ..., 1670.339478,
       1670.677612, 1671.902222], dtype=object)}
keys list [3, 71, 127, 182, 223, 291, 357, 425, 486, 573, 592, 623, 709, 780, 848, 925, 983, 1009, 1080, 1097, 1129, 1158, 1215, 1292, 1361, 1434, 1555, 1652, 1679, 1684]
keys list [7, 59, 177, 242, 311, 375, 446, 504, 505, 538, 560, 585, 656, 731, 800, 869, 947, 1015, 1037, 1048, 1069, 1130, 1175, 1237, 1313, 1384, 1525, 1628, 1638, 1673]
[[3, 127, 223, 357, 486, 592, 709, 848, 983], [1009, 1097, 1158], [1215, 1361]]
[[177, 311, 446, 505, 560, 656, 800, 947], [1175, 1313]]
[[(7, 59)], [(177, 242), (242, 311), (311, 375), (375, 446), (446, 504), (504, 505), (505, 538), (538, 560), (560, 585), (585, 656), (656, 731), (731, 800), (800, 869), (

# Double Support Time, a feature of PD patients

In [ ]:
def separate_cycles(LHEE_combined_list, RHEE_combined_list, LTOE_combined_list, RTOE_combined_list):
    L_gait_boundaries = []
    R_gait_boundaries = []

    # make a list of ranges (also lists) marking the boundaries of each full cycle
    # for LHEE strikes
    # this will not combine two LHEE heel strikes from separate lists within combined_list
    # no connections across missing/removed steps will be made
    for i in range(0, len(LHEE_combined_list) - 1):
        current_list = LHEE_combined_list[i]
        if len(current_list) == 1:
            continue
        else:
            for n in range(0, len(current_list) - 1):
                hs_initial = current_list[n]
                hs_next = current_list[n + 1]
                L_gait_boundaries.append([hs_initial, hs_next])
    # print(L_gait_boundaries)
    
    # for RHEE strikes
    for i in range(0, len(RHEE_combined_list) - 1):
        current_list = RHEE_combined_list[i]
        if len(current_list) == 1:
            continue
        else:
            for n in range(0, len(current_list) - 1):
                hs_initial = current_list[n]
                hs_next = current_list[n + 1]
                R_gait_boundaries.append([hs_initial, hs_next])           
    # print(R_gait_boundaries)

    # list of (L IC 1, R IC 1, L IC2) cycles
    cycle_bounds = []

    # check for right heel strike frames that come in between boundaries
    # add between the initial and subsequent left heel strike frames
    for i in range(0, len(L_gait_boundaries)):
        current_L_cycle = L_gait_boundaries[i]
        L_bound_1 = current_L_cycle[0]
        L_bound_2 = current_L_cycle[1]

        for n in range(0, len(R_gait_boundaries)):
            test_R_cycle = R_gait_boundaries[n]
            R_bound_1 = test_R_cycle[0]
            R_bound_2 = test_R_cycle[1]

            if L_bound_1 < R_bound_1 < L_bound_2:
                cycle_bounds.append((L_bound_1, R_bound_1, L_bound_2))

            if L_bound_1 < R_bound_2 < L_bound_2:
                cycle_bounds.append((L_bound_1, R_bound_2, L_bound_2))

    
    cycle_bounds = list(set(cycle_bounds))
    cycle_bounds = list(sorted(cycle_bounds))

    # make each gait cycle its own dictionary
    # the keys of each dictionary are gait events (heel strike, toe-off)
    # exact keys (for full dict): 'LHS1', 'RTO', 'RHS', 'LTO', LHS2'
    cycles = []
    cycles_dict = {}
    for i in range(0, len(cycle_bounds)):
        cycles_dict['LHS1'] = cycle_bounds[i][0]
        cycles_dict['RHS'] = cycle_bounds[i][1]
        cycles_dict['LHS2'] = cycle_bounds[i][2]
        cycles.append(cycles_dict)
        cycles_dict = {}
  

    return cycles

# include toe-off events in the dictionaries if their values are
# between their surrounding heel strike events
def toe_events(cycles, LTOE_combined_list, RTOE_combined_list):
    L_toe_offs = []
    R_toe_offs = []
    gait_cycles = []

    if len(LTOE_combined_list) == 1:
        current_list = LTOE_combined_list[0]
        if len(current_list) > 1:
            for n in range(0, len(current_list) - 1):
                to_initial = current_list[n]
                to_next = current_list[n + 1]
                L_toe_offs.append([to_initial, to_next])
    else:
        for i in range(0, len(LTOE_combined_list)):
            current_list = LTOE_combined_list[i]
            if len(current_list) == 1:
                continue
            else:
                for n in range(0, len(current_list) - 1):
                    to_initial = current_list[n]
                    to_next = current_list[n + 1]
                    L_toe_offs.append([to_initial, to_next])

    if len(RTOE_combined_list) == 1:
        current_list = RTOE_combined_list[0]
        if len(current_list) > 1:
            for n in range(0, len(current_list) - 1):
                to_initial = current_list[n]
                to_next = current_list[n + 1]
                R_toe_offs.append([to_initial, to_next])
    else:
        for i in range(0, len(RTOE_combined_list)):
            current_list = RTOE_combined_list[i]
            if len(current_list) > 1:
                for n in range(0, len(current_list) - 1):
                    to_initial = current_list[n]
                    to_next = current_list[n + 1]
                    R_toe_offs.append([to_initial, to_next])

    # print(R_toe_offs)

    for i in range(0, len(cycles)):
        current_cycle = cycles[i]
        # insert right TO values between left initial and right HS
        for n in range(0, len(R_toe_offs)):
            RTO1 = R_toe_offs[n][0]
            RTO2 = R_toe_offs[n][1]

            if current_cycle['LHS1'] < RTO1 < current_cycle['RHS']:
                current_cycle['RTO'] = RTO1
            if current_cycle['LHS1'] < RTO2 < current_cycle['RHS']:
                current_cycle['RTO'] = RTO2

        # insert left TO values into cycles
        for m in range(0, len(L_toe_offs)):
            LTO1 = L_toe_offs[m][0]
            LTO2 = L_toe_offs[m][1]

            if current_cycle['RHS'] < LTO1 < current_cycle['LHS2']:
                current_cycle['LTO'] = LTO1
            if current_cycle['RHS'] < LTO2 < current_cycle['LHS2']:
                current_cycle['LTO'] = LTO2
        
  
        gait_cycles.append(current_cycle)

    print(gait_cycles)

    return gait_cycles

# only include gait cycles with all HS and TO events in calculations
def filter_gc(gait_cycles):
    full_gait_cycles = []
    for i in range(0, len(gait_cycles)):
        if len(gait_cycles[i].keys()) == 5:
            full_gait_cycles.append(gait_cycles[i])
    
    print(full_gait_cycles)

    return full_gait_cycles

# calculates how long each gait cycle is (in frames)
def gait_cycle_times(full_gait_cycles):
    cycle_times = []

    for cycle in full_gait_cycles:
        cycle_time = cycle['LHS2'] - cycle['LHS1']
        cycle_times.append(cycle_time)
    
    # uncomment below to define gait cycles starting with R foot heel strike
    # for cycle in cycles:
    #     cycle_time = cycle['RHS2'] - cycle['RHS1']
    #     cycle_times.append(cycle_time)

    cycle_times = np.array(cycle_times)
    # print(cycle_times)

    cycle_times_clean = cd.remove_outliers(cycle_times)
    # print(cycle_times_clean)
    # print(np.average(cycle_times_clean))

    return cycle_times, cycle_times_clean

def support_time(full_gait_cycles):
    ds_times = []
    ss_times_L = []
    ss_times_R = []
    
    for i in range(0, len(full_gait_cycles)):
        # calculate single support times for each leg
        current_cycle = full_gait_cycles[i]
        L_single = current_cycle['RHS'] - current_cycle['RTO']
        R_single = current_cycle['LHS2'] - current_cycle['LTO']
        # uncomment below if gait cycles start with RHS
        # R_single = current_cycle['LHS'] - current_cycle['LTO']
        # L_single = current_cycle['RHS2'] - current_cycle['RTO']
        current_cycle['L_ss'] = L_single
        current_cycle['R_ss'] = R_single

        ss_times_L.append(L_single)
        ss_times_R.append(R_single)

        # calculate double support time for the cycle
        double = (current_cycle['RTO'] - current_cycle['LHS1']) + (current_cycle['LTO'] - current_cycle['RHS'])
        # double = (current_cycle['LTO'] - current_cycle['RHS1']) + (current_cycle['RTO'] - current_cycle['LHS'])
        current_cycle['ds'] = double

        ds_times.append(double)
    
    # add single and double support times into arrays
    ss_times_L = np.array(ss_times_L)
    ss_times_R = np.array(ss_times_R)
    ds_times = np.array(ds_times)

    return ss_times_L, ss_times_R, ds_times

In [ ]:
cycles = separate_cycles(LHEE_combined_list, RHEE_combined_list, LTOE_combined_list, RTOE_combined_list)
print(cycles)
#print('LHEE', LHEE_combined_list)
#print('RHEE', RHEE_combined_list)
#print('LTOE', LTOE_combined_list)
#print('RTOE', RTOE_combined_list)
gait_cycles = toe_events(cycles, LTOE_combined_list, RTOE_combined_list)
#print(gait_cycles)
full_gait_cycles = filter_gc(gait_cycles)
print('full gait cycles,' ,full_gait_cycles) #5 markers in each section 

[{'LHS1': 3, 'RHS': 71, 'LHS2': 127}, {'LHS1': 127, 'RHS': 182, 'LHS2': 223}, {'LHS1': 223, 'RHS': 291, 'LHS2': 357}, {'LHS1': 357, 'RHS': 425, 'LHS2': 486}, {'LHS1': 486, 'RHS': 573, 'LHS2': 592}, {'LHS1': 592, 'RHS': 623, 'LHS2': 709}, {'LHS1': 709, 'RHS': 780, 'LHS2': 848}, {'LHS1': 848, 'RHS': 925, 'LHS2': 983}, {'LHS1': 1009, 'RHS': 1080, 'LHS2': 1097}, {'LHS1': 1097, 'RHS': 1129, 'LHS2': 1158}]
[{'LHS1': 3, 'RHS': 71, 'LHS2': 127}, {'LHS1': 127, 'RHS': 182, 'LHS2': 223}, {'LHS1': 223, 'RHS': 291, 'LHS2': 357, 'RTO': 242, 'LTO': 311}, {'LHS1': 357, 'RHS': 425, 'LHS2': 486, 'RTO': 375, 'LTO': 446}, {'LHS1': 486, 'RHS': 573, 'LHS2': 592}, {'LHS1': 592, 'RHS': 623, 'LHS2': 709, 'LTO': 656}, {'LHS1': 709, 'RHS': 780, 'LHS2': 848, 'RTO': 731, 'LTO': 800}, {'LHS1': 848, 'RHS': 925, 'LHS2': 983, 'RTO': 869, 'LTO': 947}, {'LHS1': 1009, 'RHS': 1080, 'LHS2': 1097, 'RTO': 1069}, {'LHS1': 1097, 'RHS': 1129, 'LHS2': 1158}]
[{'LHS1': 223, 'RHS': 291, 'LHS2': 357, 'RTO': 242, 'LTO': 311}, {'LHS1

# Joint Angles (crediting [source](https://www.mdpi.com/1424-8220/22/19/7178) for equation )

In [ ]:
import math

# Define a function to calculate the angle between two vectors
#define what the angles are
def calculate_angle(vector1, vector2):
    dot_product = sum([vector1[i]*vector2[i] for i in range(len(vector1))])
    vector1_length = math.sqrt(sum([v**2 for v in vector1]))
    vector2_length = math.sqrt(sum([v**2 for v in vector2]))
    cos_theta = dot_product / (vector1_length * vector2_length)
    return math.acos(cos_theta)

# Define the vectors constituting the angles
vector_AB = [1, 0, 0]   # Vector from A to B
vector_BC = [0, 1, 0]   # Vector from B to C
vector_CD = [0, 0, 1]   # Vector from C to D
vector_DE = [0, 1, 1]   # Vector from D to E
vector_EF = [1, 0, 1]   # Vector from E to F
vector_FG = [1, 1, 0]   # Vector from F to G

# Calculate the angles (hip, knee, ankle joint)
angle_ABC = calculate_angle(vector_AB, vector_BC)
angle_CDE = calculate_angle(vector_CD, vector_DE)
angle_EFG = calculate_angle(vector_EF, vector_FG)

# Print the results
print(f"Angle ABC: {math.degrees(angle_ABC)} degrees")
print(f"Angle CDE: {math.degrees(angle_CDE)} degrees")
print(f"Angle EFG: {math.degrees(angle_EFG)} degrees")


Angle ABC: 90.0 degrees
Angle CDE: 45.00000000000001 degrees
Angle EFG: 60.00000000000001 degrees


# Box and Whisker Data Visualization for Hip and Knee angles (4 subplots swing, stance for each marker) [source](https://www.mdpi.com/1424-8220/22/19/7178) 

In [ ]:
anvil.server.wait_forever()